In [ ]:
# pip install imgaug

In [1]:
import random
import numpy as np
import os
import shutil
import cv2
import imgaug as ia
import imgaug.augmenters as iaa
import matplotlib.pyplot as plt
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage
from PIL import Image
import PIL.ImageOps
# imgaug의 시드 고정
ia.seed(42)

In [2]:
pwd

'C:\\Users\\user\\Desktop\\recognition_project'

In [3]:
%cd "./yolov5/archive"

C:\Users\user\Desktop\recognition_project\yolov5\archive


In [4]:
def route(t):
    file_path = os.path.join("./images", t)
    pth = []
    for f in os.listdir(file_path):
        pth.append(os.path.join(file_path, f))
    return pth
pth_tr = route("train")
pth_val = route("valid")

In [5]:
def annot_route(t):
    file_path = os.path.join("./labels", t)
    pth = []
    for f in os.listdir(file_path):
        pth.append(os.path.join(file_path, f))
    return pth
annot_tr = annot_route("train")
annot_val = annot_route("valid")

In [6]:
d_set = [[pth_tr, annot_tr], [pth_val, annot_val]]

### Data Augmentation 하기 전 파악해야 할 사항
- Rotation 
1. annotation 파일 저장 시에 정규화를 진행했는가?
    - 진행했음
    - affine transform 하기 전에 다시 복구 시켜서 진행할 것
    - x_center의 경우는 image의 width로 나눴다(image.size[0])
    - y_center의 경우는 image의 height로 나눴다(image.size[1])
2. x1, y1, x2, y2는 각각 어떤 것인가?
    - x1의 경우는 이미지의 x축의 좌측 끝, x2는 우측 끝
    - y1은 이미지의 y축의 가장 아래, y2는 가장 위

In [16]:
def make_noise(std, img):
    height, width, ch = img.shape
    # 각 channel에 적용
    # 하나 더 추가
    # 빈 배열 형성
    img_noise = np.zeros((height, width, ch), dtype = np.float)
    for i in range(height):
        for j in range(width):
            for k in range(ch):
                # gaussian distribution을 따르는 random 숫자를 넣는다
                mk_noise = np.random.normal()
                set_noise = std * mk_noise
                noise_input = img[i][j][k] + set_noise
                if noise_input > 255:
                    noise_input = 255
                if noise_input < 0:
                    noise_input = 255
                img_noise[i][j][k] = noise_input
    return img_noise

In [19]:
def data_augmentation(random_n, img, annot, n):
    image = cv2.imread(img)
    img_pth = img
    img_annot = annot
    # rotate
    if random_n == 1:
        # annotation file을 읽는다.
        f = open(img_annot)
        bbx = f.readlines()
        # normalize된 것을 복구하기 위한 작업
        d_w = image.shape[0] # 폭
        d_h = image.shape[1] # 높이
        # 하나씩 읽어온다.
        class_no = []
        for i in range(len(bbx)):
            bbxs = bbx[i].split(" ")
            for j in range(len(bbxs)):
                # 이는 class에 해당한다.
                if j == 0:
                    bbxs[j] = int(bbxs[j])
                    class_no.append(int(bbxs[j]))
                # str 형태를 float으로 바꿔온다
                else:
                    x = float(bbxs[j])
                    bbxs[j] = x
            # 대체하는 작업
            bbx[i] = bbxs
        # 작업해야할 bounding box
        bbx_new = []
        for i in range(len(bbx)):
            # 원래의 x_center
            x_center = bbx[i][1] * d_w
            # 원래의 y_center
            y_center = bbx[i][2] * d_h
            # 원래의 폭
            w = bbx[i][3] * d_w
            # 원래의 높이
            h = bbx[i][4] * d_h
            # x1, x2는 각각 boundingbox의 x좌표의 양 끝단
            # y1, y2는 각각 boundingbox의 y좌표의 양 끝단
            x1 = int(x_center - w/2)
            y1 = int(y_center - h/2)
            x2 = int(x_center + w/2)
            y2 = int(y_center + h/2)
            bbx_new.append([x1, y1, x2, y2])
        # 정규화 -> 원 상태
        # 여기에 이제 rotation을 적용
        # ia_bbxs는 우선 data augmentation을 적용시키기 위한 사전 작업
        ia_bbxs = []
        for b in bbx_new:
            ia_bbxs.append(BoundingBox(x1 = b[0], y1 = b[1], x2 = b[2], y2 = b[3]))
        bound_box = BoundingBoxesOnImage(ia_bbxs, shape = image.shape)
        # 180도 회전을 다음과 같이 진행한다.
        seq = iaa.Sequential([iaa.Affine(rotate = 180)])
        # 180도 회전 적용
        img_aug, bbx_aug = seq(image = image, bounding_boxes = bound_box)
        # 이미지를 각 파일에 저장
        storage_dir = os.path.join(img_pth[:14], "rotation"+str(n)+".png")
        cv2.imwrite(storage_dir, img_aug)
        # annotation file 역시 저장
        # 똑같이 정규화를 시켜서 저장한다.
        annot_file_path = os.path.join(img_annot[:14], "rotation"+str(n)+".txt")
        with open(annot_file_path, "w") as f:
            for a in range(len(bbx_aug)):
                class_id = class_no[a]
                # opencv의 경우는 높이, 폭, 채널 순서
                dx = 1/int(image.shape[1])
                dy = 1/int(image.shape[0])
                x1 = bbx_aug[a][0][0]
                x2 = bbx_aug[a][1][0]
                y1 = bbx_aug[a][0][1]
                y2 = bbx_aug[a][1][1]
                x_center = (x1+x2)/2 * dx
                y_center = (y1+y2)/2 * dy
                w = (x2-x1) * dx
                h = (y2-y1) * dy
                f.write(f"{class_id} {x_center:.5f} {y_center:.5f} {w:.5f} {h:.5f}\n")
    # Noise 추가
    # 기본적으로 noise로는 Gaussian noise를 추가한다.
    else:
        noise_img = make_noise(12, image)
        storage_dir = os.path.join(img_pth[:14], "noise"+str(n)+".png")
        # 이미지 저장
        cv2.imwrite(storage_dir, noise_img)
        # annotation 저장
        new_annot_file_path = os.path.join(img_annot[:14], "noise"+str(n)+".txt")
        original_annot_path = img_annot
        shutil.copyfile(original_annot_path, new_annot_file_path)

In [ ]:
for idx in range(len(d_set)):
    for n in range(len(d_set[idx][0])):
        random_num = random.randrange(1, 3)
        data_augmentation(random_num, d_set[idx][0][n], d_set[idx][1][n], n)

C:\Users\user\AppData\Local\Temp\ipykernel_9588\3718389932.py:6: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  img_noise = np.zeros((height, width, ch), dtype = np.float)
